# PyTorchを使った転移学習

## 転移学習

- **転移学習**
    - 学習済みのモデルの層の一部を付け替えて、新しいパラメータを学習させるディープラーニング手法の一つ
    - 一から学習させる場合に比べて少ない教師データと時間で学習させることができる
- 学習済みモデルの使い方
    - 基本的に現在学習済みモデルとして公開されているものは、ほぼ全てPythonフレームワークで作られたものである
    - DeepLearningモデルを様々なフレームワーク間で交換するためのフォーマットとして**ONNX**(オニキス)形式が提唱されている
        - JuliaのネイティブDeepLearningフレームワーク「Flux」用にONNXモデルをインポートするライブラリもある
        - 現時点では、まだ開発途中で完全にONNXモデルをロードすることはできない
    - Juliaのフレームワーク等が充実するまではPyCallを介してPyTorchなどのフレームワークを使うのが良いかもしれない

In [1]:
include("./lib/Image.jl")
include("./lib/TorchVision.jl")
using .TorchVision

In [2]:
using Random

# 乱数初期化
## Random.seed!([rng=GLOBAL_RNG], seed) -> rng
## Random.seed!([rng=GLOBAL_RNG]) -> rng
### `!`付きの関数は第一引数の値を破壊的に変更する
Random.seed!(1234)

# PyTorchの乱数初期化
torch.manual_seed(1234)

PyObject <torch._C.Generator object at 0x7f1bd7cd8c30>

In [3]:
using PyCall

# 訓練用、予測用の画像変換関数を作成する関数
## () -> ((PyObject, String) -> Array{Float32,3})
make_transformer_for_learning() = begin
    resize = 224
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    transform = Dict(
        "train" => make_transformer(
            transforms.RandomResizedCrop(resize; scale=(0.5, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.Normalize(mean, std)
        ),
        "val" => make_transformer(
            transforms.Resize(resize),
            transforms.CenterCrop(resize),
            transforms.Normalize(mean, std)
        )
    )
    return (image::PyObject; phase::String="train") -> transform[phase](image)
end

image_transform_vgg16 = make_transformer_for_learning()

#3 (generic function with 1 method)

In [4]:
# ハリネズミとヤマアラシの画像へのファイルパスのリスト作成
make_dataset_list(phase::String="train") = begin
    hedgehogs = map(
        path -> "./dataset/$(phase)/hedgehog/$(path)",
        readdir("./dataset/$(phase)/hedgehog/")
    )
    porcupines = map(
        path -> "./dataset/$(phase)/porcupine/$(path)",
        readdir("./dataset/$(phase)/porcupine/")
    )
    vcat(hedgehogs, porcupines)
end

train_list = make_dataset_list("train")

585-element Array{String,1}:
 "./dataset/train/hedgehog/118523311_32345c36a2.jpg"    
 "./dataset/train/hedgehog/1241612498_7ab4277d10.jpg"   
 "./dataset/train/hedgehog/126009980_9004803c9e.jpg"    
 "./dataset/train/hedgehog/1274493397_88388552d8.jpg"   
 "./dataset/train/hedgehog/127772208_f65a074ed5.jpg"    
 "./dataset/train/hedgehog/1295991716_4ad47dae66.jpg"   
 "./dataset/train/hedgehog/1296287640_19d39d5b1e.jpg"   
 "./dataset/train/hedgehog/1322807353_6eec9596b3.jpg"   
 "./dataset/train/hedgehog/150464690_e33dd1938d.jpg"    
 "./dataset/train/hedgehog/159959475_fb41beb469.jpg"    
 "./dataset/train/hedgehog/163878245_fd30b5169b.jpg"    
 "./dataset/train/hedgehog/17404099_32851ad117.jpg"     
 "./dataset/train/hedgehog/176380875_d2ad991223.jpg"    
 ⋮                                                      
 "./dataset/train/porcupine/PA210066.JPG"               
 "./dataset/train/porcupine/porcupine_sc108.jpg"        
 "./dataset/train/porcupine/porcupine_sud_america.jpg"  
 "

In [5]:
# ハリネズミとヤマアラシのデータセット作成
@pydef mutable struct Dataset <: torch.utils.data.Dataset
    __init__(self, phase::String="phase") = begin
        pybuiltin(:super)(Dataset, self).__init__()
        self.phase = phase
        self.file_list = make_dataset_list(phase)
    end
    
    __len__(self) = length(self.file_list)
    
    __getitem__(self, index::Int) = begin
        # index番目の画像をロード
        ## Juliaのindexは1〜なので +1 する
        img_path = self.file_list[index + 1]
        img = Image.open(img_path)
        img_transformed = image_transform_vgg16(img; phase=self.phase)
        # 画像のラベル名をパスから抜き出す
        label = img_path[length(self.phase) + 12 : length(self.phase) + 19]
        # ハリネズミ: 0, ヤマアラシ: 1
        label = (label == "hedgehog" ? 0 : 1)
        return img_transformed, label
    end
end

train_dataset = Dataset("train")
val_dataset = Dataset("val")

# 動作確認
index = 0
img_transformed, label = train_dataset.__getitem__(index)

(Float32[-2.1179 -2.1179 … -2.1179 -2.1179; -2.03571 -2.03571 … -2.03571 -2.03571; -1.80444 -1.80444 … -1.80444 -1.80444]

Float32[-2.1179 -2.1179 … -2.1179 -2.1179; -2.03571 -2.03571 … -2.03571 -2.03571; -1.80444 -1.80444 … -1.80444 -1.80444]

Float32[-2.1179 -2.1179 … -2.1179 -2.1179; -2.03571 -2.03571 … -2.03571 -2.03571; -1.80444 -1.80444 … -1.80444 -1.80444]

...

Float32[0.536433 0.964552 … -0.422553 0.211063; 0.170168 0.590336 … -0.302521 0.345238; 0.0604794 0.531068 … -0.0789542 0.565926]

Float32[1.01593 0.930302 … 0.211063 1.80366; 0.677871 0.590336 … 0.345238 1.97339; 0.548497 0.409063 … 0.565926 2.18684]

Float32[0.656306 0.570682 … 1.18717 -0.0458088; 0.310224 0.257703 … 1.34314 0.0826331; 0.112767 -0.0789542 … 1.55939 0.304488], 0)

In [6]:
# ミニバッチサイズ
batch_size = 32

# DataLoader作成
train_dataloader = torch.utils.data.DataLoader(
    train_dataset; batch_size=batch_size, shuffle=true
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset; batch_size=batch_size, shuffle=true
)

# 辞書にまとめる
dataloaders = Dict(
    "train" => train_dataloader,
    "val" => val_dataloader
)

Dict{String,PyObject} with 2 entries:
  "val"   => PyObject <torch.utils.data.dataloader.DataLoader object at 0x7f1b9…
  "train" => PyObject <torch.utils.data.dataloader.DataLoader object at 0x7f1b9…

In [7]:
# 学習済みVGG-16モデルをロード
net = models.vgg16(pretrained=true)

# VGG-16の最後の全結合出力層の出力ユニットを2個に付け替える
## 出力は ハリネズミ=0, ヤマアラシ=1 の2種類分類
net.classifier[7] = torch.nn.Linear(in_features=4096, out_features=2)

# 訓練モードに設定
net.train()

┌ Warning: `setindex!(o::PyObject, v, i::Integer)` is deprecated, use `set!(o, i - 1, v)` instead.
│   caller = top-level scope at In[7]:3
└ @ Core In[7]:3


PyObject VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17

In [8]:
# 損失関数の定義
criterion = torch.nn.CrossEntropyLoss()

# 転移学習で学習させるパラメータを params_to_update に格納
params_to_update = []

# 学習させるパラメータ名
update_param_names = ["classifier.6.weight", "classifier.6.bias"]

# 学習させるパラメータ以外は勾配計算させない
for (name, param) in net.named_parameters()
    if in(name, update_param_names)
        param.required_grad = true
        push!(params_to_update, param)
        println(name)
    else
        param.required_grad = false
    end
end

# params_to_updateの中身を確認
println("----------")
println(params_to_update)

classifier.6.weight
classifier.6.bias
----------
Any[PyObject Parameter containing:
tensor([[-0.0109,  0.0036, -0.0132,  ...,  0.0019,  0.0018, -0.0121],
        [-0.0107,  0.0017,  0.0034,  ...,  0.0134,  0.0052, -0.0079]],
       requires_grad=True), PyObject Parameter containing:
tensor([-0.0060,  0.0011], requires_grad=True)]


In [9]:
# 最適化手法の設定
optimizer = torch.optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)

PyObject SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)

In [10]:
# モデル訓練
train_model(net, dataloaders, criterion, optimizer, num_epochs) = begin
    tqdm = pyimport("tqdm").tqdm
    
    # epoch数分ループ
    for epoch = 1:num_epochs
        println("Epoch $(epoch)/$(num_epochs)")
        println("----------")
        
        # epochごとの学習と検証のループ
        for phase in ["train", "val"]
            if phase == "train"
                net.train() # 訓練モードに
            else
                net.eval() # 検証モードに
            end
            
            epoch_loss = 0.0 # epochの損失和
            epoch_corrects = 0 # epochの正解数
            
            # 未学習時の検証性能を確かめるため、最初の訓練は省略
            if epoch == 1 && phase == "train"
                continue
            end
            
            # データローダーからミニバッチを取り出すループ
            ## tqdmによるプログレスバーは、Julia＋JupyterNotebookではリアルタイム描画されないため、正直意味はない
            for (inputs, labels) in tqdm(dataloaders[phase])
                # optimizer初期化
                optimizer.zero_grad()
                
                # 順伝搬計算
                torch.set_grad_enabled(phase == "train")
                outputs = net(inputs)
                loss = criterion(outputs, labels) # 損失計算
                (max, preds) = torch.max(outputs, 1) # ラベルを予測
                # 訓練時はバックプロパゲーション
                if phase == "train"
                    loss.backward()
                    optimizer.step()
                end
                # イテレーション結果の計算
                epoch_loss += loss.item() * inputs.size(0)
                epoch_corrects += torch.sum(preds == labels.data)
                torch.set_grad_enabled(false)
            end
            
            # epochごとの損失と正解率を表示
            epoch_loss = epoch_loss / length(dataloaders[phase].dataset)
            epoch_acc = epoch_corrects^2 / length(dataloaders[phase].dataset)
            println("$(phase) Loss: $(epoch_loss), Acc: $(epoch_acc)")
        end
    end
end

# 学習・検証を実行
train_model(net, dataloaders, criterion, optimizer, 2)

Epoch 1/2
----------


 33%|██████████████▋                             | 1/3 [00:00<00:00,  5.73it/s]

PyCall.PyError: PyError (ccall(#= /home/user/.julia/packages/PyCall/ttONZ/src/pyiterator.jl:81 =# @pysym(:PyIter_Next), PyPtr, (PyPtr,), s[2])) <class 'RuntimeError'>
RuntimeError("output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]")
  File "/home/user/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tqdm/_tqdm.py", line 937, in __iter__
    for obj in iterable:
  File "/home/user/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 560, in __next__
    batch = self.collate_fn([self.dataset[i] for i in indices])
  File "/home/user/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 560, in <listcomp>
    batch = self.collate_fn([self.dataset[i] for i in indices])
  File "PyCall", line 1, in <lambda>
  File "/home/user/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torchvision/transforms/transforms.py", line 61, in __call__
    img = t(img)
  Fi

#### RuntimeError("output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]") について
これは、グレースケール画像が混ざっているために起こるエラーである

本来は、グレースケールの画像を探し出して削除するのが良いのだが、面倒なので、画像読み込み時にRGB画像として読み込むように変更する

In [11]:
# ハリネズミとヤマアラシのデータセット作成
## ※ 画像をRGB画像として読み込む
@pydef mutable struct Dataset <: torch.utils.data.Dataset
    __init__(self, phase::String="phase") = begin
        pybuiltin(:super)(Dataset, self).__init__()
        self.phase = phase
        self.file_list = make_dataset_list(phase)
    end
    
    __len__(self) = length(self.file_list)
    
    __getitem__(self, index::Int) = begin
        # index番目の画像をロード
        ## Juliaのindexは1〜なので +1 する
        img_path = self.file_list[index + 1]
        img = Image.open(img_path).convert("RGB") # ←追加
        img_transformed = image_transform_vgg16(img; phase=self.phase)
        # 画像のラベル名をパスから抜き出す
        label = img_path[length(self.phase) + 12 : length(self.phase) + 19]
        # ハリネズミ: 0, ヤマアラシ: 1
        label = (label == "hedgehog" ? 0 : 1)
        return img_transformed, label
    end
end

train_dataset = Dataset("train")
val_dataset = Dataset("val")

# DataLoader作成
train_dataloader = torch.utils.data.DataLoader(
    train_dataset; batch_size=batch_size, shuffle=true
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset; batch_size=batch_size, shuffle=true
)

# 辞書にまとめる
dataloaders = Dict(
    "train" => train_dataloader,
    "val" => val_dataloader
)

# 学習・検証を実行
train_model(net, dataloaders, criterion, optimizer, 2)

Epoch 1/2
----------
val Loss: 0.6778282999992371, Acc: PyObject tensor(23)
Epoch 2/2
----------



100%|████████████████████████████████████████████| 3/3 [00:08<00:00,  2.13s/it]

train Loss: 0.3890837358103858, Acc: PyObject tensor(407)



100%|██████████████████████████████████████████| 19/19 [03:24<00:00, 11.29s/it]

val Loss: 0.3321733415126801, Acc: PyObject tensor(61)



100%|████████████████████████████████████████████| 3/3 [00:07<00:00,  1.96s/it]

In [12]:
# 転移学習したモデルで改めてハリネズミ画像を認識させる

net.eval() # 推論モードに設定

# 画像読み込み
image_file_path = "./data/gahag-0059907781-1.jpg"
img = Image.open(image_file_path)

# 画像をVGG16に読み込ませられるように処理する
transform = make_transformer_for_vgg16()
img_transformed = transform(img)

# 転移学習したVGG-16モデルで予測実行
pred = predict(net, [img_transformed])

1×2 Array{Float32,2}:
 -1.14885  1.74748

ラベルは `[ハリネズミ, ヤマアラシ]` と定義したため、上記の予測は `ヤマアラシ` という結果を表している

したがって、今回の転移学習は失敗したということができる

In [15]:
# ヤマアラシの画像でも予測してみる
img2 = Image.open("./data/publicdomainq-0025120muq.jpg")
img2_transformed = transform(img2)
pred = predict(net, [img2_transformed])

1×2 Array{Float32,2}:
 -1.42318  1.77903

In [16]:
# 転移学習したモデルのパラメータを保存する
torch.save(net.state_dict(), "./vgg16_weight.pth")

## 結果と考察

今回は、上手く転移学習させることができず、ハリネズミとヤマアラシを識別するモデルを作成することはできなかった

この原因としては以下のようなものが考えられる

1. ハリネズミとヤマアラシの教師データの数に差がありすぎた
    - 以下のように、ヤマアラシの画像はハリネズミの画像の5倍近くあり、学習には不向きだった
        - 訓練用画像数:
            - ハリネズミ:  98枚
            - ヤマアラシ: 487枚
        - 検証用画像数:
            - ハリネズミ: 40枚
            - ヤマアラシ: 40枚
2. 教師データそのものが誤っている可能性があった
    - 人間が手動で分類しており、教師データそのものの妥当性が割と怪しかった
3. 教師データ量が足りていなかった
4. そもそもVGG-16モデル自体古いモデルであり、精度がそれほど高くない